# Un semplice modello linguistico

Costruiamo uno "N-gram Language Model" allenandolo sui testi di Sanremo. Partiamo dal modello più semplice e usiamo bigrammi e trigrammi.

Un'ottima guida per costruire un semplice modello del genere attraverso l'utilissima libreria [NLTK](https://www.nltk.org/) può essere consultata [qui](https://www.nltk.org/api/nltk.lm.html).

In [ ]:
from nltk.corpus import PlaintextCorpusReader
from nltk.lm import MLE
from nltk.lm.preprocessing import padded_everygram_pipeline
import nltk
import os

## Caricare il corpus

Per prima cosa carichiamo il corpus. Usiamo l'ottima libreria [NLTK]() per gestire i molti file in cui il corpus è diviso

In [ ]:
root = os.path.expanduser('~/Documents/sync/data/Sanremo_testi')
os.path.isdir(root)


In [ ]:
corpus_sanremo = PlaintextCorpusReader(root, r'.*\.txt')
len(corpus_sanremo.fileids())

Il corpus contiene 244 file.

Ora, da ciascuno di loro, estraiamo il testo ed eseguiamo alcune comuni operazioni di preparazione:

* eseguiamo il *sentence splitting* e la tokenizzaione (NLTK lo farà per noi)
* eliminiamo i segni di punteggiatura
* convertiamo tutte le maiuscole in minuscole

In [ ]:
sents = []
for s in corpus_sanremo.sents():
    toks = [t.lower() for t in s if t.isalnum()]
    sents.append(toks)

Alcune statistiche di base sul nostro corpus

In [ ]:
print(f'There are {len(sents)} sentences in the corpus; \nThere are {sum([len(s) for s in sents])} tokens in the corpus')


## Addestriamo il nostro modello

### Padding e bigrammi

`NLTK.lm` mette a nostra disposizione uno strumento molto utile che permette di fare 3 cose molto facilmente:

- inserire due token speciali (`<s>` e `</s>`) per l'inizio e fine di frase
- calcolare tutti gli $n$-grammi fino ad un massimo dato (es. 3 per unigrammi, bigrammi e trigrammi)
- definire il nostro vocabolario (l'insieme di tutte le forme usate)

In [ ]:
train, vocab = padded_everygram_pipeline(3, sents)

### Addestriamo i modelli

Per il bigram model:

In [ ]:
bi_lm = MLE(2)
bi_lm.fit(train, vocab)
print(bi_lm.vocab)

In [ ]:
train, vocab = padded_everygram_pipeline(3, sents)

lm = MLE(3)
lm.fit(train, vocab)
print(lm.vocab)

## Esploriamo i modelli

### Contare i bigrammi (trigram)

In [ ]:
lm.counts[['il', 'mio']].N()

In [ ]:
lm.counts[['un', 'brutto']].most_common(5)

In [ ]:
lm.counts[['il', 'mio']].most_common(5)

In [ ]:
lm.counts['amore']

### Probabilità

Qual è la probabilità di una parola $W$ data una sequenza? Vediamo alcune probabilità del modello a trigrammi 

In [ ]:
lm.score('amore', ['il', 'mio'])

Ricordatevi che, secondo la LME, la probabilità di un trigramma si calcola secondo la formula

$P(amore|il\;mio) = \frac{f(\text{il mio amore})}{f(\text{il mio })}$

In [ ]:
gens = lm.generate(10, text_seed=['amore', 'mio'], random_seed=3)
print(' '.join(gens))

Qual è la probabilità di "mio" se la parola che precede è "amore"?

Qual è la probabilità di "cuore" preceduto da "mio"?

In [ ]:
lm.score('cuore', ['il', 'mio'])

La freq di "il mio" è:

In [ ]:
f_il_mio = lm.counts[['il', 'mio']].N()
print(f_il_mio)

La freq di "il mio amore" è:

In [ ]:
f_il_mio_amore = lm.counts[['il', 'mio']]['amore']
print(f_il_mio_amore)

Quali sono le 5 parole più probabili dopo un dato bigramma?

In [ ]:
def get_top_5_most_probable_words(lm, bigram):
    word_probs = {word: lm.score(word, bigram) for word in lm.vocab}
    return sorted(word_probs.items(), key=lambda x: x[1], reverse=True)[:5]

In [ ]:
get_top_5_most_probable_words(lm, ['dammi', 'un'])

## Generare testo!

Possiamo usare il nostro modello per generare del testo dato un contesto iniziale (es. un bigramma)

In [ ]:
lm.generate(10, text_seed=['il', 'mio'])

## Un generatore più sofisticato

In [ ]:
from nltk.tokenize.treebank import TreebankWordDetokenizer

detokenize = TreebankWordDetokenizer().detokenize

def generate_from_seed(model, seed_words, num_words=10, random_seed=42):
    """
    Generate text starting from a given seed word.
    
    :param model: An ngram language model from `nltk.lm.model`.
    :param seed_words: The initial words to start generation.
    :param num_words: Max number of words to generate.
    :param random_seed: Seed value for randomness.
    :return: A generated sentence as a string.
    """
    content = list(seed_words)  # Start with the seed word
    context = tuple(seed_words)  # Context for the bigram model

    for _ in range(num_words - 1):  # -1 because seed word is already added
        next_word = model.generate(1, text_seed=context, random_seed=random_seed)
        if next_word == '</s>':  # Stop at sentence end
            break
        content.append(next_word)
        context = (next_word,)  # Update context

    return detokenize(content)


In [ ]:
# generate_from_seed(lm, ['il', 'mio'], num_words=10, random_seed=32)
generate_from_seed(lm, ['un', 'bel'], num_words=15, random_seed=4)

### Ancora più sofisticato

Aumentiamo la randomizzazione

In [ ]:
from nltk.tokenize.treebank import TreebankWordDetokenizer
import random

detokenize = TreebankWordDetokenizer().detokenize

def generate_from_seed_trigram(model, seed_words, max_length=100, temperature=0.8):
    """
    Generate text from a trigram model with a stochastic selection mechanism to avoid loops.

    :param model: A trained trigram language model from `nltk.lm.model`.
    :param seed_words: A tuple or list containing exactly two seed words.
    :param max_length: Maximum number of words to generate (default: 100).
    :param temperature: A parameter that controls randomness (higher = more randomness).
    :return: A generated sentence as a string.
    """
    if not isinstance(seed_words, (list, tuple)) or len(seed_words) != 2:
        raise ValueError("seed_words must be a list or tuple containing exactly two words.")

    content = list(seed_words)  # Start with the two given words
    context = tuple(seed_words)  # Initialize context for the trigram model

    for _ in range(max_length - 2):  # -2 because we already have two words
        possible_next_words = list(model.context_counts(context).keys())
        
        if not possible_next_words:  # No next word found, stop generation
            break

        # Get probabilities of next words
        probabilities = [model.score(w, context) for w in possible_next_words]

        # Apply softmax-like temperature scaling
        probabilities = [p ** (1 / temperature) for p in probabilities]
        total_prob = sum(probabilities)
        probabilities = [p / total_prob for p in probabilities]

        # Sample next word based on probabilities
        next_word = random.choices(possible_next_words, weights=probabilities, k=1)[0]

        if next_word == '</s>':  # Stop at end-of-sentence token
            break

        content.append(next_word)
        context = (context[1], next_word)  # Update context with the last two words

    return detokenize(content)


In [ ]:
print(generate_from_seed_trigram(lm, 
                         ['si', 'inghiottì'],
                         max_length=50,
                         temperature=1.5
                         ))

## Appendice: vedere le concordanze

In [ ]:
from nltk.text import Text

conc = Text(corpus_sanremo.words())

In [ ]:
conc.concordance(['amore', 'che'], width=100, lines=40)